# This is a notebook for Solving PDEs as Petri Nets with PyCIEMSS

### Load dependencies

In [1]:
import os
import pyciemss

### Set simulation parameters

In [2]:
start_time = 0.0
end_time = 100.0
logging_step_size = 10.0
num_samples = 3

# Advection equation

### Load models from [DARPA-ASKEM/Model-Representations](https://github.com/DARPA-ASKEM/Model-Representations/tree/pde-petri-amrs/petrinet/examples/pde/advection)

In [3]:
# MODEL_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/pde-petri-amrs/petrinet/examples/pde/advection/"
MODEL_PATH = "../../docs/source/"

advection1D1 = os.path.join(MODEL_PATH, "advection_backward_1_0.01_3.json")
advection1D2 = os.path.join(MODEL_PATH, "advection_backward_1_0.01_5.json")
advection1D3 = os.path.join(MODEL_PATH, "advection_backward_1_0.01_10.json")

### Sample from model

In [4]:
result = pyciemss.sample(advection1D1, end_time, logging_step_size, num_samples, start_time=start_time)
result["unprocessed_result"]

ERROR:root:
                ###############################

                There was an exception in pyciemss

                Error occured in function: sample

                Function docs : 
    Load a model from a file, compile it into a probabilistic program, and sample from it.

    Args:
        model_path_or_json: Union[str, Dict]
            - A path to a AMR model file or JSON containing a model in AMR form.
        end_time: float
            - The end time of the sampled simulation.
        logging_step_size: float
            - The step size to use for logging the trajectory.
        num_samples: int
            - The number of samples to draw from the model.
        solver_method: str
            - The method to use for solving the ODE. See torchdiffeq's `odeint` method for more details.
            - If performance is incredibly slow, we suggest using `euler` to debug.
              If using `euler` results in faster simulation, the issue is likely that the model is s

RuntimeError: The size of tensor a (9) must match the size of tensor b (3) at non-singleton dimension 0
                                       Trace Shapes:    
                                        Param Sites:    
        numeric_initial_state_func$$$_nodes.0._value    
        numeric_initial_state_func$$$_nodes.1._value    
        numeric_initial_state_func$$$_nodes.2._value    
numeric_deriv_func$$$_nodes.0._args.0._args.0._value    
                                       Sample Sites:    
                                  persistent_dx dist 3 |
                                               value 3 |
                                   persistent_u dist 3 |
                                               value 3 |